In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as trans
import matplotlib
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
train_set = dsets.MNIST(
    root='./data', 
    train=True, 
    transform=trans.ToTensor(), 
    download=True)

In [4]:
print('Number of images {}'.format(len(train_set)))
print('Type {}'.format(type(train_set[0][0])))
print('size of images {}'.format(train_set[0][0].size()))

Number of images 60000
Type <class 'torch.Tensor'>
size of images torch.Size([1, 28, 28])


In [5]:
class Model5_1(nn.Module):
    def __init__(self):
        self.inSize=28
        self.hiddenSize=100
        self.numLayers = 2
        self.outSize = 10
        super(Model5_1, self).__init__()
        self.rnn = nn.RNN(self.inSize, 
                          self.hiddenSize, 
                          self.numLayers, 
                          batch_first=True)
        self.fc = nn.Linear(self.hiddenSize, self.outSize)
        
    def forward(self, x):
        h0 = torch.zeros(self.numLayers, x.size(0), self.hiddenSize).to(device)
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, 1, :])
        return out

In [7]:
model = Model5_1().cuda()
model.to(device)

lr = 0.001
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

In [8]:
batchSize = 128
trainloader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=batchSize,
    num_workers = 12,
    pin_memory=True,
    shuffle=True)

In [9]:
epochs = 5
for epoch in range(epochs):
    runningLoss = 0.0
    for i, (images, labels) in enumerate(trainloader):
        optimizer.zero_grad()
        images = images.view(-1, 28, 28)
        images = images.to(device)
        labels = labels.to(device)
        out = model(images).to(device)
        loss = loss_function(out, labels)
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()
    print('epoch {}, loss {}'.format(epoch, runningLoss))

epoch 0, loss 1077.5509405136108
epoch 1, loss 1075.8177480697632
epoch 2, loss 1075.6397306919098
epoch 3, loss 1075.5641367435455
epoch 4, loss 1075.4994173049927


In [11]:
import numpy as np
def successRate(predicted, labels):
    predict = [np.argmax(p.cpu().detach().numpy()) for p in predicted]
    actual =  [labels[i].item() for i in range(len(predicted))]
    correct = [i for i, j in zip(predict, actual) if i == j]
    return(len(correct)/(len(predict)))

predicted = model.forward(images) 
predicted.size()
successRate(predicted, labels)

0.11458333333333333

In [12]:
testSet = dsets.MNIST(root='./data', train = False, transform=trans.ToTensor(), download=True)
testloader = torch.utils.data.DataLoader(dataset=testSet,  batch_size = 10000, shuffle = True)

testData = iter(testloader)
images, labels = testData.next()
images = images.view(-1, 28, 28)
images = images.to(device)
labels = labels.to(device)
output = model.forward(images)
successRate(output,labels)

0.1071

In [17]:
torch.save(model.state_dict(), 'rnnmodel.pkl')
model.load_state_dict(torch.load('rnnmodel.pkl'))